# Data Loading, Cleaning and Transforming

## Libraries

In [90]:
import pandas as pd
import numpy as np
import os
import deepl
import config

## Data Installing and Cleaning

In [91]:
car_columns = "p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,p10,p11,p12,p13a,p13b,p13c,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p27,p28,p34,p35,p39,p44,p45a,p47,p48a,p49,p50a,p50b,p51,p52,p53,p55a,p57,p58,a,b,d,e,f,g,h,i,j,k,l,n,o,p,q,r,s,t,p5a".split(',')
pedestrian_col = ['p1','p29','p30','p31','p32']

drop_cols = ['weekday(p2a)','a', 'b', 'd', 'e','f', 'g', 'j', 'l', 'n', 'o', 'p', 'q', 'r', 's', 't']

Unfortunately the documents didn't say what these letters meant, so some are still a mystery to me

In [92]:
files_path = '../data/raw/'

In [93]:
car_data = []
pedestrian_data = []

folders = os.listdir(files_path)
folders = list(filter(lambda x: x.find('.') != 0,folders))

for folder in folders:
  folder_path = files_path + folder + '/'
  csvs = os.listdir(folder_path)
  for csv in csvs:
    csv_path = folder_path+csv
# create chunks for each csv file(will be faster)
    if csv == 'CHODCI.csv':
      chodci_chunk = pd.read_csv(csv_path,sep=';',
                                  names=pedestrian_col,
                                  encoding='Windows-1250',
                                  dtype=str,chunksize=10000)
      [pedestrian_data.append(chunk) for chunk in chodci_chunk]
    else:
      chunks = pd.read_csv(csv_path, sep=';',
                            names=car_columns,
                            encoding='Windows-1250',
                            dtype=str, chunksize=10000)
      [car_data.append(chunk) for chunk in chunks]

df = pd.concat(car_data,ignore_index=True)
df_pedestrian = pd.concat(pedestrian_data,ignore_index=True)

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707027 entries, 0 to 707026
Data columns (total 64 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   p1            707027 non-null  object
 1   p36           707027 non-null  object
 2   p37           329366 non-null  object
 3   p2a           707027 non-null  object
 4   weekday(p2a)  707027 non-null  object
 5   p2b           707027 non-null  object
 6   p6            707027 non-null  object
 7   p7            707027 non-null  object
 8   p8            707027 non-null  object
 9   p9            707027 non-null  object
 10  p10           707027 non-null  object
 11  p11           707027 non-null  object
 12  p12           707027 non-null  object
 13  p13a          707027 non-null  object
 14  p13b          707027 non-null  object
 15  p13c          707027 non-null  object
 16  p14           707027 non-null  object
 17  p15           707025 non-null  object
 18  p16           707025 non

In [95]:
df = df.drop(drop_cols,axis=1)

In [96]:
df.duplicated().sum()

4

In [97]:
df.head(3)

,p1,p36,p37,p2a,p2b,p6,p7,p8,p9,p10,...,p51,p52,p53,p55a,p57,p58,h,i,k,p5a
0,031017220679,1,21,2022-11-24,1954,3,0,6,2,1,...,1,1,200,2,1,1,TRSTĚNICE,NaN,silnice 1.třídy,2
1,031017220700,1,21,2022-11-24,1954,1,2,0,2,1,...,1,1,1000,2,1,1,CHODOVÁ PLANÁ,NaN,silnice 1.třídy,2
2,190206220001,3,21413,2022-01-01,0530,9,0,0,1,1,...,1,3,400,3,5,0,NaN,NaN,silnice3.třídy,2


## Transform categorical values

In [98]:
original_nan = df.isna().mean().to_frame().reset_index()

In [99]:
original_nan[original_nan[0] > 0]

,index,0
2,p37,0.534154
16,p15,0.000003
17,p16,0.000003
18,p17,0.000003
19,p18,0.000003
20,p19,0.000003
21,p20,0.000003
30,p39,0.883502
32,p45a,0.172107
33,p47,0.044799


Look at the original missing values so that nothing is lost when changing them

In [100]:
transformation_path = '../data/transformation_data/'

In [101]:
cat_values = pd.read_csv(f'{transformation_path}cat_names.csv',sep=';',encoding='Windows-1250')
cat_values.loc[cat_values['Bližší definice'].notna(),'Popis'] = cat_values['Popis'] + ' ' + cat_values['Bližší definice']
cat_values = cat_values.drop('Bližší definice',axis=1)
cat_values = cat_values[~cat_values['Formulář DN'].isin(['p47','p53','p37'])]

In [102]:
target_language = "en"
translator = deepl.Translator(config.DEEPL_API_KEY)

In [103]:
cat_values['Popis'] = cat_values['Popis'].apply(lambda x: translator.translate_text(str(x),target_lang='EN-US').text)

Exceptions

In [104]:
def grouping(row):
  if float(row) in range(11,19):
    return '11-18'
  elif float(row) in range(22,29):
    return '22-28'
  else:
    return row

In [105]:
df['p35'] = df['p35'].apply(grouping)
df['p52'] = df['p52'].apply(lambda x: '10-99' if float(x) in range(10,100) else x)

In [106]:
normalized_values = {'00':'0','01':'1','02':'2','03':'3','04':'4','05':'5','06':'6','07':'7','08':'8','09':'9'}

for col in df.columns:
    if col in cat_values['Formulář DN'].values:
#p35 is an exception, you don't need to generalize there.
        if col != 'p35' and df[col].isin(normalized_values).any():
            df[col] = df[col].replace(normalized_values)
#get our values for our column and translate that into a key value
        temp = cat_values[cat_values['Formulář DN'] == col]
        mapping_dict = dict(zip(temp['Položka'], temp['Popis']))
        df[col] = df[col].map(mapping_dict).astype('category')
df.head(3)

,p1,p36,p37,p2a,p2b,p6,p7,p8,p9,p10,...,p51,p52,p53,p55a,p57,p58,h,i,k,p5a
0,031017220679,Class 1 roads,21,2022-11-24,1954,collision with a fixed obstacle,not an option It is not a collision between mo...,"wall, fixed part of bridges, underpasses, tunn...",accident with material damage only,driver of a motor vehicle,...,there was no need to use violence,moving vehicle - in the measure of stationary ...,200,with a driving licence of category B,good no adverse circumstances were found,the driver was not affected,TRSTĚNICE,NaN,silnice 1.třídy,outside the village
1,031017220700,Class 1 roads,21,2022-11-24,1954,collision with a moving non-rail vehicle,Side,not an option not a collision with a fixed obs...,accident with material damage only,driver of a motor vehicle,...,there was no need to use violence,moving vehicle - in the measure of stationary ...,1000,with a driving licence of category B,good no adverse circumstances were found,the driver was not affected,CHODOVÁ PLANÁ,NaN,silnice 1.třídy,outside the village
2,190206220001,3rd class roads,21413,2022-01-01,0530,crashes,not an option It is not a collision between mo...,not an option not a collision with a fixed obs...,fatal accident,driver of a motor vehicle,...,there was no need to use violence,vehicle moving - against the direction of stat...,400,with a driving licence of category C,"under the influence of alcohol, blood alcohol ...",other influences,NaN,NaN,silnice3.třídy,outside the village


In [107]:
new_nan = df.isna().mean().to_frame().reset_index()

Checking for mistakes

In [108]:
result = pd.merge(new_nan,original_nan,how='left',left_on='index',right_on='index')
result[result['0_x'] != result['0_y']]

,index,0_x,0_y
16,p15,0.002977,0.000003
31,p44,0.002059,0.000000
32,p45a,0.212594,0.172107
34,p48a,0.181013,0.175467
39,p52,0.175442,0.170059
42,p57,0.215272,0.205018


There are no errors in the code just some values have omissions in their excel file.  
There is no way to recognize these values

In [109]:
new_nan[new_nan[0] > 0.3]

,index,0
2,p37,0.534154
30,p39,0.883502
44,h,0.322940
45,i,0.445825


h and i are city and street, so we can't get rid of them, the rest are unlikely to be used at all

In [110]:
new_cols = ['id','communication_kind','date','time','accident_kind','crash_kind','fixed_precaution_kind','injury',
            'cause_of_accident','alcohol','main_cause_of_accident','killed_persons','severely_injured_persons','slightly_injured_persons','total_material_damage',
            "road_surface_type", "condition_of_vehicle_surface", "condition_of_communication", "weather_conditions_at_the_time_of_accident", "visibility", "visibility_ratios",
            'communication_division', 'communication_accident_situation', 'traffic_control_at_the_time_of_an_accident', 'local_administration_priority_in_traffic',
            'specific_location_and_objects_in_the_location_of_the_accident','directional_rates','number_of_vehicles_involved','location_of_the_traffic_accident',
            'vehicle_type','vehicle_brand','year_of_manufacture','vehicle_owner','skid','vehicle_sequel',
            'leakage_of_transported_materials','way_of_releasing_persons_from_the_vehicle','direction_of_movement','damage_to_the_vehicle(in_hundred_crowns)','driver_category',
            'driver_condition','driver_external_influence','city','street','road_type','location']

In [111]:
df = df.drop(['p37','p39'],axis=1)
df.columns = new_cols

In [112]:
df.head(3)

,id,communication_kind,date,time,accident_kind,crash_kind,fixed_precaution_kind,injury,cause_of_accident,alcohol,...,way_of_releasing_persons_from_the_vehicle,direction_of_movement,damage_to_the_vehicle(in_hundred_crowns),driver_category,driver_condition,driver_external_influence,city,street,road_type,location
0,031017220679,Class 1 roads,2022-11-24,1954,collision with a fixed obstacle,not an option It is not a collision between mo...,"wall, fixed part of bridges, underpasses, tunn...",accident with material damage only,driver of a motor vehicle,ne,...,there was no need to use violence,moving vehicle - in the measure of stationary ...,200,with a driving licence of category B,good no adverse circumstances were found,the driver was not affected,TRSTĚNICE,NaN,silnice 1.třídy,outside the village
1,031017220700,Class 1 roads,2022-11-24,1954,collision with a moving non-rail vehicle,Side,not an option not a collision with a fixed obs...,accident with material damage only,driver of a motor vehicle,ne,...,there was no need to use violence,moving vehicle - in the measure of stationary ...,1000,with a driving licence of category B,good no adverse circumstances were found,the driver was not affected,CHODOVÁ PLANÁ,NaN,silnice 1.třídy,outside the village
2,190206220001,3rd class roads,2022-01-01,0530,crashes,not an option It is not a collision between mo...,not an option not a collision with a fixed obs...,fatal accident,driver of a motor vehicle,yes blood alcohol content between 1.0 ‰ and 1.5 ‰,...,there was no need to use violence,vehicle moving - against the direction of stat...,400,with a driving licence of category C,"under the influence of alcohol, blood alcohol ...",other influences,NaN,NaN,silnice3.třídy,outside the village


## Changing types and filling missing values

In [113]:
df['date'] = pd.to_datetime(df['date'])


In [114]:
df['time'] = df['time'].astype(int)
df['damage_to_the_vehicle(in_hundred_crowns)'] = df['damage_to_the_vehicle(in_hundred_crowns)'].fillna('-1').astype(int)
df['total_material_damage'] = df['total_material_damage'].astype(int)

cons_cols = ['killed_persons','severely_injured_persons','slightly_injured_persons']
df[cons_cols] = df[cons_cols].astype(int)

In [115]:
df['injury'] = df['injury'].map({'accident with material damage only':0,'fatal accident':1}).astype(int)

In [116]:
road_translate = pd.DataFrame(df['road_type'].unique())
road_translate['new'] = road_translate[0].apply(lambda x: translator.translate_text(str(x),target_lang='EN-US').text)

In [117]:
new_type = dict(road_translate.values)

In [118]:
df['road_type'] = df['road_type'].map(new_type)

In [119]:
df.loc[df['road_type'] == 'nan'] = np.nan

In [120]:
def year_to_int(row):
  if type(row) == float or row in ['XX','--','AN']:
    return -1
  elif  23 > int(row) > 0:
    return 2000+ int(row)
  else:
    return 1900+int(row)

In [121]:
df['year_of_manufacture'].unique()

array(['04', '11', '99', '93', 'XX', '08', '16', '06', '15', '10', '21',
       '12', '01', '19', '02', '20', '17', '00', '07', '13', '18', nan,
       '22', '05', '96', '09', '97', '14', '03', '95', '98', '86', '94',
       '91', 'AN', '87', '92', '89', '--', '85', '90', '80', '88', '77',
       '84', '74', '72', '53', '79', '44', '24', '81', '82', '69', '83',
       '0', '48', '65', '57', '62', '61', '64', '75', '56', '66', '27',
       '67', '63', '23', '52', '58', '76', '55', '71', '59', '28', '42',
       '78', '73', '38', '70', '36', '54', '34', '60', '68', '40', '32',
       '47', '49', '39', '50', '45'], dtype=object)

In [122]:
df['year_of_manufacture'] = df['year_of_manufacture'].apply(year_to_int).astype('int16')

In [123]:
df.isna().mean()

id                                                               0.008964
communication_kind                                               0.008964
date                                                             0.008964
time                                                             0.008964
accident_kind                                                    0.008964
crash_kind                                                       0.008964
fixed_precaution_kind                                            0.008964
injury                                                           0.008964
cause_of_accident                                                0.008964
alcohol                                                          0.008964
main_cause_of_accident                                           0.008964
killed_persons                                                   0.008964
severely_injured_persons                                         0.008964
slightly_injured_persons              

In [124]:
df[df['vehicle_type'] == 'undetected, driver drove off'].isna().mean()

id                                                               0.000000
communication_kind                                               0.000000
date                                                             0.000000
time                                                             0.000000
accident_kind                                                    0.000000
crash_kind                                                       0.000000
fixed_precaution_kind                                            0.000000
injury                                                           0.000000
cause_of_accident                                                0.000000
alcohol                                                          0.000000
main_cause_of_accident                                           0.000000
killed_persons                                                   0.000000
severely_injured_persons                                         0.000000
slightly_injured_persons              

видим что 17 процентов пропущенных значений это когда водитель уехал с места происшествия

In [125]:
col_filtr = df[df['vehicle_type'] == 'undetected, driver drove off'].isna().mean() >= 0.98
cols = df.columns[col_filtr]
cols

Index(['vehicle_brand', 'vehicle_owner', 'skid', 'vehicle_sequel',
       'leakage_of_transported_materials',
       'way_of_releasing_persons_from_the_vehicle', 'direction_of_movement',
       'driver_category', 'driver_condition', 'driver_external_influence'],
      dtype='object')

In [126]:
df[cols] = df[cols].astype(str)
df.loc[df['vehicle_type'] == 'undetected, driver drove off',cols] = 'undetected, driver drove off'
df[cols] = df[cols].astype('category')

In [127]:
df.isna().mean()

id                                                               0.008964
communication_kind                                               0.008964
date                                                             0.008964
time                                                             0.008964
accident_kind                                                    0.008964
crash_kind                                                       0.008964
fixed_precaution_kind                                            0.008964
injury                                                           0.008964
cause_of_accident                                                0.008964
alcohol                                                          0.008964
main_cause_of_accident                                           0.008964
killed_persons                                                   0.008964
severely_injured_persons                                         0.008964
slightly_injured_persons              

In [132]:
df = df[df['id'].notna()].copy()

In [133]:
print(df['number_of_vehicles_involved'].unique())
df['number_of_vehicles_involved'] = df['number_of_vehicles_involved'].astype('int16')

['1' '2' '01' '02' '03' '05' '04' '3' '4' '5' '6' '06' '07' '8' '09' '08'
 '7' '11' '20' '10' '17' '25' '15' '12' '14']


In [135]:
df.to_csv('../data/processed/road_accidents_czechia_2016_2022.csv',index=False)

# Pedestrian

In [136]:
df_pedestrian.head()

,p1,p29,p30,p31,p32
0,000030226275,1,1,3,10
1,000030228673,2,1,3,06
2,000030229624,3,1,1,04
3,000032220930,2,1,5,03
4,000033225303,3,2,1,04


In [137]:
df_pedestrian = df_pedestrian.dropna()
df_pedestrian = df_pedestrian.astype(int)

In [138]:
pedestrian_values = pd.read_csv(f'{transformation_path}pedestrian_col_values.csv',sep=';')
pedestrian_values = pedestrian_values.fillna('')
pedestrian_values['description'] = pedestrian_values['description'] + ' ' + pedestrian_values['wider']
pedestrian_values['description'] = pedestrian_values['description'].str.strip()

pedestrian_values.tail()

,column,value,description,wider
29,p32,7,"chůze, stání na chodníku",stání na chodníku
30,p32,8,chůze po správné straně,po správné straně
31,p32,9,chůze po nesprávné straně,po nesprávné straně
32,p32,10,přecházení mimo přechod (20 a více metrů od p...,mimo přechod (20 a více metrů od přechodu)
33,p32,0,jiná situace,


In [139]:
pedestrian_values['description'] = pedestrian_values['description'].apply(lambda x: translator.translate_text(str(x),target_lang='EN-US').text)

In [140]:
for col in df_pedestrian.columns:
  if col in pedestrian_values['column'].values:
    map_values = dict(pedestrian_values[pedestrian_values['column'] == col][['value','description']].values)
    df_pedestrian[col] = df_pedestrian[col].map(map_values)

In [141]:
df_pedestrian.head()

,p1,p29,p30,p31,p32
0,30226275,man,good no adverse circumstances were found,sudden entry into the roadway from the sidewal...,Crossing outside the crossing (20 metres or mo...
1,30228673,Woman,good no adverse circumstances were found,sudden entry into the roadway from the sidewal...,crossing just in front of or behind a parked v...
2,30229624,child (up to 15 years),good no adverse circumstances were found,"correct, appropriate",crossing at a marked crossing
3,32220930,Woman,good no adverse circumstances were found,"confused, hasty, indecisive behaviour",pedestrian entering the roadway near the cross...
4,33225303,child (up to 15 years),"inattention, distraction","correct, appropriate",crossing at a marked crossing


In [143]:
df_pedestrian.columns = ['id','pedestrian_category','pedestrian_condition',
                         'pedestrian_behaviour','situation_at_the_point_of_accident']

In [144]:
df_pedestrian.to_csv('../data/processed/pedestrian.csv',index=False)